In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from LSTM_model import LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [77]:
file_path = '../../data/dataS.csv'
df = pd.read_csv(file_path, parse_dates=['Date'])
#df = df.dropna()
#df = df[df['volatility_forcast'] <= 0.2]
#df = df[df['volatility'] <= 0.15]

df.head()

,Date,Open,High,Low,Close,SP500_Adj_Close,move,Volume,volatility,volatility_forcast,returns,GARCH_VM
0,2003-01-03,0.0477,0.0030,0.0015,0.0005,0.0482,1,0.0698,0.0668,0.2115,0.5843,0.0472
1,2003-01-06,0.0476,0.0014,0.0027,0.0049,0.0525,0,0.0973,0.2115,0.0912,0.6219,0.0377
2,2003-01-07,0.0518,0.0030,0.0009,0.0006,0.0512,0,0.1071,0.0912,0.1286,0.5742,0.0319
3,2003-01-08,0.0505,0.0033,0.0002,0.0020,0.0485,1,0.1001,0.1286,0.1648,0.5617,0.0256
4,2003-01-09,0.0478,0.0005,0.0028,0.0044,0.0522,0,0.1085,0.1648,0.1331,0.6169,0.0212


In [78]:
target = 'volatility_forcast'
date='Date'

df=df.drop(columns=[date])
X = df.drop(columns=[target])
y = df[target]


In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
input_size = X_train.shape[1]  # Number of features per time step
hidden_size = 64  # Example hidden layer size
output_size = y_train.shape[1] if len(y_train.shape) > 1 else 1  # Adjust based on your target

In [81]:
X_train_np = X_train.to_numpy()  # or X_train.values
y_train_np = y_train.to_numpy()  # or y_train.values

In [82]:
# Create an instance of the model
lstm_model = LSTM(input_size, hidden_size, output_size)

# Train the model (ensure that X_train and y_train are formatted as a sequence)
lstm_model.train(X_train_np, y_train_np, learning_rate=0.01, epochs=100)

e:\ESIEA\Illinois Institute of Technology\MATH 594 Research Project\Research_Project\code_\models\LSTM_model.py:114: RuntimeWarning: overflow encountered in add
  dz = (np.dot(self.W_f.T, df) +
e:\ESIEA\Illinois Institute of Technology\MATH 594 Research Project\Research_Project\code_\models\LSTM_model.py:96: RuntimeWarning: overflow encountered in add
  dc = dh * o_t * tanh_derivative(tanh(c_t)) + dc_next
e:\ESIEA\Illinois Institute of Technology\MATH 594 Research Project\Research_Project\code_\models\LSTM_model.py:106: RuntimeWarning: overflow encountered in add
  dW_c += np.dot(d_c_tilde, z.T)
e:\ESIEA\Illinois Institute of Technology\MATH 594 Research Project\Research_Project\code_\models\LSTM_model.py:111: RuntimeWarning: overflow encountered in add
  db_c += d_c_tilde


Epoch 0, Loss: 8.209278573204212
Epoch 10, Loss: nan
Epoch 20, Loss: nan
Epoch 30, Loss: nan
Epoch 40, Loss: nan
Epoch 50, Loss: nan
Epoch 60, Loss: nan
Epoch 70, Loss: nan
Epoch 80, Loss: nan
Epoch 90, Loss: nan


In [83]:
# Predict using the forward pass (X_test should also be in the same sequence format)
y_pred, hidden_states, cell_states = lstm_model.forward(X_test)


KeyError: 0